In [2]:
import tkinter as tk
from tkinter import filedialog
from tkinter import *
from PIL import ImageTk, Image
import numpy as np
from tensorflow.keras.models import load_model
import cv2
import os, shutil
from glob import glob
import math
import tqdm
import tensorflowjs as tfjs
import numpy
from tensorflow.keras.preprocessing import image
from sklearn.metrics import accuracy_score
import sys
from tensorflow.keras.applications import EfficientNetB0
import efficientnet.keras as efn
import re
import youtube_dl
from flask import Flask, jsonify, request
import time
import pyautogui

app=Flask(__name__)
print("Awaiting Video Player")
model=load_model(r'/home/kashish/Desktop/FECD/NPDIv3fullofull.12-0.17.h5')
saved_url=""
url_queue=[]
tab_queue=[]
flag_queue=[]

def url_changed(saved_url):
    wait.until(
        lambda driver: driver.current_url!=saved_url)
    return True;

def url_strip(url):
    if "http://" in url or "https://" in url:
        url = url.replace("https://", '').replace("http://", '').replace('\"', '')
    if "/" in url:
        url = url.split('/', 1)[0]
    return url

def predict():
    while len(url_queue):
        video_url=url_queue.pop(0)
        prediction_images=[]
        print("Video Player found")
        start = time.time()
        ydl_opts={}
        ydl=youtube_dl.YoutubeDL(ydl_opts)
        info_dict=ydl.extract_info(video_url, download=False)
        formats=info_dict.get('formats',None)

        print("Obtaining frames")
        for f in formats:
            if f.get('format_note',None) == '144p':
                url = f.get('url',None)
                cap = cv2.VideoCapture(url)
                if not cap.isOpened():
                    print('video not opened')
                    exit(-1)
                x=0
                count=0
                while x<10:
                    ret, frame = cap.read()
                    if not ret:
                        break
                    filename =r"/home/kashish/Desktop/FECD/temp/shot"+str(x)+".png"
                    x+=1
                    cv2.imwrite(filename.format(count), frame)
                    count+=100
                    cap.set(1,count)
                    if cv2.waitKey(30)&0xFF == ord('q'):
                        break
                cap.release()

        print(str(x)+" frames obtained successfully")
        print("Running Prediction Model")

        images = glob(r"/home/kashish/Desktop/FECD/temp/*.png")

        for i in range(len(images)):
            img = image.load_img(images[i], target_size=(224,224,3))
            img = numpy.array(img)
            img = img/255
            prediction_images.append(img)

        prediction_images=numpy.array(prediction_images)
        pred=model.predict(prediction_images)
        y_pred = numpy.argmax(pred, axis=-1)

        for i in range(0,len(y_pred)):
            if y_pred[i]==1 and pred[i][1]<=0.95:
                y_pred[i]=0
        print(numpy.sum(y_pred),math.ceil(10*len(prediction_images)/100))
        confiporn=[]
        confinonporn=[]
        for i in range(0,len(y_pred)):
            if y_pred[i]==1:
                confiporn.append(pred[i][1])
            else:
                confinonporn.append(pred[i][0])
        confiporn=numpy.array(confiporn)
        confinonporn=numpy.array(confinonporn)

        if(numpy.sum(y_pred)>=math.ceil(10*len(prediction_images)/100)):
            prediction=1
            confidence=numpy.mean(confiporn)
            flag_queue.append(tab_queue.pop(0))
        else:
            prediction=0
            confidence=numpy.mean(confinonporn)
            tab_queue.pop(0)
        confi = numpy.round(confidence*100)

        end=time.time()
        print("Predicted to be of type "+str(prediction)+" in "+str(round(end-start,3))+" seconds") 

        print("...for "+url)
        folder=r"/home/kashish/Desktop/FECD/temp"
        for filename in os.listdir(folder):
            file_path = os.path.join(folder, filename)
            try:
                if os.path.isfile(file_path) or os.path.islink(file_path):
                    os.unlink(file_path)
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)
            except Exception as e:
                print('Failed to delete %s. Reason: %s' % (file_path, e))

@app.route('/send_url', methods=['POST'])
def send_url():
    resp_json = request.get_data()
    params = resp_json.decode()
    print("Obtaining new URL")
    url=params.split(',', 1)[0]
    url=url.replace("url=",'')
    tabId=params.split(',', 1)[1]
    tabId=tabId.replace("tab=",'')
    parent_url=url_strip(url)
    print(parent_url)
    if(parent_url=="www.youtube.com" and "watch" in url):
        print("New video player found")
        url_queue.append(url)
        tab_queue.append(tabId)
        if len(url_queue)==1:
            predict()
    else:
        print("Video player either not found or not supported for this path")
    return jsonify({'message': 'success!'}), 200
    
@app.route('/quit_url', methods=['POST'])
def quit_url():
    resp_json = request.get_data()
    print("Url closed: " + resp_json.decode())
    return jsonify({'message': 'quit success!'}), 200    

@app.route('/blur_player',methods=['GET'])
def blur_player():
    return jsonify(flag_queue);

app.run(host='127.0.0.1', port=5000)

Awaiting Video Player
ERROR! Session/line number was not unique in database. History logging moved to new session 9
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


Obtaining new URL
https://www.youtube.com/watch?v=W-TE_Ys4iwM
134
www.youtube.com
New video player found
Video Player found
[youtube] W-TE_Ys4iwM: Downloading webpage
Obtaining frames
10 frames obtained successfully
Running Prediction Model


127.0.0.1 - - [15/Apr/2021 14:01:47] "POST /quit_url HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Apr/2021 14:01:47] "POST /quit_url HTTP/1.1" 200 -
ERROR:__main__:Exception on /send_url [POST]
Traceback (most recent call last):
  File "/home/kashish/anaconda3/lib/python3.8/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/kashish/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/kashish/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/home/kashish/anaconda3/lib/python3.8/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/home/kashish/anaconda3/lib/python3.8/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/kashish/anaconda3/lib/python3.8/site-packages/flask/app

0 1
Predicted to be of type 0 in 7.6 seconds
Url closed: url=https://www.youtube.com/watch?v=W-TE_Ys4iwM
Found and closed flagged content
...for https://r2---sn-gwpa-a3vy.googlevideo.com/videoplayback?expire=1618497100&ei=7Pl3YMS6IOyHjuMPntGz2AU&ip=2405%3A201%3A5003%3Ab03b%3A1a77%3Af546%3A614%3A2ea5&id=o-AKA5wi-XJPEt7DA3OjNuwaCqNuLgysQ7UEY3pgUDTTzl&itag=278&aitags=133%2C134%2C135%2C136%2C137%2C160%2C242%2C243%2C244%2C247%2C278%2C394%2C395%2C396%2C397%2C398%2C399&source=youtube&requiressl=yes&mh=7D&mm=31%2C29&mn=sn-gwpa-a3vy%2Csn-qxaeen7e&ms=au%2Crdu&mv=m&mvi=2&pl=38&gcr=in&initcwndbps=883750&vprv=1&mime=video%2Fwebm&ns=tHykjFEFkBjAWfDlmeWjM00F&gir=yes&clen=2700666&dur=247.640&lmt=1579864408664193&mt=1618475073&fvip=2&keepalive=yes&fexp=24001373%2C24007246&beids=9466587&c=WEB&txp=5535432&n=Tt25nX9YNEMlo7k&sparams=expire%2Cei%2Cip%2Cid%2Caitags%2Csource%2Crequiressl%2Cgcr%2Cvprv%2Cmime%2Cns%2Cgir%2Cclen%2Cdur%2Clmt&lsparams=mh%2Cmm%2Cmn%2Cms%2Cmv%2Cmvi%2Cpl%2Cinitcwndbps&lsig=AG3C_xAwRQI

...

Obtaining new URL
https://www.google.co.in/search?q=switch+to+a+particular+chrome+tab+python+without+selenium&sxsrf=ALeKk03LrrEDdWAUVND0HacRxtVXPYTcGg%3A1618475856613&ei=UPt3YNb-JNK6rQHk5b7wDw&oq=switch+to+a+particular+chrome+tab+python+without+selenium&gs_lcp=Cgdnd3Mtd2l6EAMyCAghEBYQHRAeOgcIABBHELADOgUIIRCgAToECCEQFVDrIlifOWClOmgBcAJ4AIAB4AGIAcIVkgEGMC4xNi4xmAEAoAEBqgEHZ3dzLXdpesgBCMABAQ&sclient=gws-wiz&ved=0ahUKEwjW5oq07P_vAhVSXSsKHeSyD_4Q4dUDCA4&uact=5
156
www.google.co.in
Video player either not found or not supported for this path


127.0.0.1 - - [15/Apr/2021 14:07:56] "POST /quit_url HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Apr/2021 14:07:56] "POST /quit_url HTTP/1.1" 200 -


Url closed: url=undefined


127.0.0.1 - - [15/Apr/2021 14:08:02] "POST /quit_url HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Apr/2021 14:08:02] "POST /quit_url HTTP/1.1" 200 -


Url closed: url=https://www.google.co.in/search?q=switch+to+a+particular+chrome+tab+python+without+selenium&sxsrf=ALeKk03LrrEDdWAUVND0HacRxtVXPYTcGg%3A1618475856613&ei=UPt3YNb-JNK6rQHk5b7wDw&oq=switch+to+a+particular+chrome+tab+python+without+selenium&gs_lcp=Cgdnd3Mtd2l6EAMyCAghEBYQHRAeOgcIABBHELADOgUIIRCgAToECCEQFVDrIlifOWClOmgBcAJ4AIAB4AGIAcIVkgEGMC4xNi4xmAEAoAEBqgEHZ3dzLXdpesgBCMABAQ&sclient=gws-wiz&ved=0ahUKEwjW5oq07P_vAhVSXSsKHeSyD_4Q4dUDCA4&uact=5
